---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)
df=pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
df.iloc[0]

'03/25/93 Total time of visit (in minutes):\n'

In [3]:
df.iloc[455]

'sHemmorage caused by probe in 1984 Medical History:\n'

In [4]:
#2009; 2010: Line 455~499 (45 entries)
a1=pd.Series(df.iloc[455:]).str.extractall(r'(?P<Date>(?P<month>\d+\b)?(?P<day>\d+\b)?(?P<year>\d{4}\b))')
a1["month"]=1
a1["day"]=1
print(a1.shape)
a1.head(5)

(45, 4)


,,Date,month,day,year
,match,,,,
455,0,1984,1,1,1984
456,0,2000,1,1,2000
457,0,2001,1,1,2001
458,0,1982,1,1,1982
459,0,1998,1,1,1998


In [5]:
#df.iloc[343:455] 6/2008; 12/2009
df.iloc[343]

'6/1998 Primary Care Doctor:\n'

In [6]:
a2=pd.Series(df.iloc[343:455]).str.extractall(r'(?P<Date>(?P<month>\d+\b)?(?P<day>\d+)?/(?P<year>\d{4}\b))')
a2.day=1
print(a2.shape)
a2.tail(5)

(112, 4)


,,Date,month,day,year
,match,,,,
450,0,1/1994,1,1,1994
451,0,12/2004,12,1,2004
452,0,3/2003,3,1,2003
453,0,7/1991,7,1,1991
454,0,7/1982,7,1,1982


In [7]:
#df.iloc[228:343] Feb 2009; Sep 2009; Oct 2010 
df.iloc[306]

'May, 2004 Hx of Brain Injury: No\n'

In [8]:
a3=pd.Series(df.iloc[228:343]).str.extractall(r'(?P<Date>(?P<month>[A-Z][a-z]+[,]?)[\s](?P<day>\d+)?(?P<year>\d{4}\b))')
a3.day=1
a3["month"]=a3["month"].apply(lambda x:x.split(",")[0][:3])
print(a3.shape)
new={
        'Jan' : 1,
        'Feb' : 2,
        'Mar' : 3,
        'Apr' : 4,
        'May' : 5,
        'Jun' : 6,
        'Jul' : 7,
        'Aug' : 8,
        'Sep' : 9, 
        'Oct' : 10,
        'Nov' : 11,
        'Dec' : 12
}
a3=a3.replace({"month": new})
a3.head(5)

(115, 4)


,,Date,month,day,year
,match,,,,
228,0,September 1985,9,1,1985
229,0,June 2011,6,1,2011
230,0,May 1986,5,1,1986
231,0,May 2016,5,1,2016
232,0,July 1977,7,1,1977


In [9]:
#df.iloc[125:194] 20 Mar 2009
df.iloc[193]

's 22 year old single Caucasian/Latino woman, unemployed Cook recent college graduate, living along with pet rabbit, with long history of depression with SA but never hospitalized, as well as obesity, hypothyroidism, and PCO, referred by new NMH PCP Dr. Evelyn Julian for urgent evaluation and treatment till first visit with colleague, Dr.  Inez Burns, on 18 Jan 1995.\n'

In [10]:
a4=pd.Series(df.iloc[125:194]).str.extractall(r'(?P<Date>(?P<day>[\d]+)[\s](?P<month>[A-Za-z]+)[\s](?P<year>\d{4}\b))')
print(a4.shape)
a4["month"]=a4["month"].apply(lambda x:x[:3])
a4=a4.replace({"month": new})
a4.head(5)

(69, 4)


,,Date,day,month,year
,match,,,,
125,0,24 Jan 2001,24,1,2001
126,0,10 Sep 2004,10,9,2004
127,0,26 May 1982,26,5,1982
128,0,28 June 2002,28,6,2002
129,0,06 May 1972,06,5,1972


In [11]:
#df.iloc[194:228] Mar 20, 2009
df.iloc[196]

".Feb 18, 1994: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.\n"

In [12]:
a5=pd.Series(df.iloc[194:228]).str.extractall(r'(?P<Date>(?P<month>[A-Za-z.]+)[\s](?P<day>[\d]+)[,]?[\s](?P<year>\d{4}\b))')
print(a5.shape)
a5["month"]=a5["month"].apply(lambda x: x.replace(".",""))
a5["month"]=a5["month"].apply(lambda x:x[:3])
a5=a5.replace({"month": new})
a5.head()

(34, 4)


,,Date,month,day,year
,match,,,,
194,0,"April 11, 1990",4,11,1990
195,0,"May 30, 2001",5,30,2001
196,0,".Feb 18, 1994",2,18,1994
197,0,"February 18, 1981",2,18,1981
198,0,"October. 11, 2013",10,11,2013


In [13]:
#04/20/2009; 04/20/09; 4/20/09; 4/3/09
df.iloc[71]

'4-13-89 Communication with referring physician?: Not Done\n'

In [14]:
a6=pd.Series(df.iloc[:125]).str.extractall(r'(?P<Date>(?P<month>\d+\b)[/-](?P<day>\d+\b)[/-](?P<year>\d{4}\b|\b\d{2}\b))')
print(a6.shape)
a6.head()

(125, 4)


,,Date,month,day,year
,match,,,,
0,0,03/25/93,03,25,93
1,0,6/18/85,6,18,85
2,0,7/8/71,7,8,71
3,0,9/27/75,9,27,75
4,0,2/6/96,2,6,96


In [15]:
a7=pd.concat([a6,a4,a5,a3,a2,a1],axis=0)
a7.iloc[99,2]=11
a7["year"]=a7["year"].apply(lambda x :"19"+str(x) if len(x)==2 else x)
a7["year"]=a7["year"].apply(lambda x:str(x))
a7["day"]=a7["day"].apply(lambda x:str(x))
a7["month"]=a7["month"].apply(lambda x:str(x))
a7["Date"]=a7["year"]+"/"+a7["month"]+"/"+a7["day"]
from datetime import datetime
a7["Date"]=a7["Date"].apply(lambda x:datetime.strptime(x, '%Y/%m/%d'))
a7['TIME'] = pd.to_datetime(a7['Date'])
a7['RANK'] = a7['TIME'].rank(ascending=False).astype("int")

In [16]:
def date_sorter():
    import numpy as np
    return np.flip(np.argsort(a7["RANK"].values),axis=0)
